In [1]:
# pip install Cartopy



In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key


In [8]:
# Load csv file created in Part I
city_data_csv = "output_data/cities.csv"
# Create dataframe
city_data_df = pd.read_csv(city_data_csv)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,81.55,89,98,4.00,EC,1681775041
1,1,mwingi,-0.9344,38.0601,64.96,97,55,4.27,KE,1681775041
2,2,sitka,57.0531,-135.3300,46.29,66,100,11.50,US,1681775042
3,3,deputatskiy,69.3000,139.9000,13.53,88,99,2.17,RU,1681775042
4,4,ola,59.5833,151.2833,30.78,72,100,8.79,RU,1681775042


In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    color = "City",
    alpha = .25
)

#Display the plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_df = city_data_df.loc[(city_data_df["Max Temp"] > 18) & (city_data_df["Max Temp"] > 25) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] < 5), :]

# Drop any rows with null values
vacation_df = vacation_df.dropna(axis = 0, how = "any")

# Display sample data
vacation_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,hermanus,-34.4187,19.2345,60.49,68,1,2.48,ZA,1681775047
20,20,bredasdorp,-34.5322,20.0403,56.93,86,2,2.44,ZA,1681775048
39,39,flinders,-34.5833,150.8552,64.18,87,1,3.00,AU,1681775055
68,68,mar del plata,-38.0023,-57.5575,58.08,89,0,2.30,AR,1681775064
87,87,salalah,17.0151,54.0924,79.14,74,0,2.28,OM,1681775072
107,107,dali,25.7000,100.1833,60.03,54,1,4.45,CN,1681775079
158,158,gwanda,-20.9333,29.0000,57.00,64,0,3.96,ZW,1681775098
163,163,chiredzi,-21.0500,31.6667,60.10,80,0,1.25,ZW,1681775101
203,203,gohpur,26.8833,93.6333,75.47,46,3,2.82,IN,1681775115
251,251,piet retief,-27.0071,30.8132,48.78,46,0,4.27,ZA,1681775133


In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df.copy().loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = np.nan

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,hermanus,ZA,-34.4187,19.2345,68,NaN
20,bredasdorp,ZA,-34.5322,20.0403,86,NaN
39,flinders,AU,-34.5833,150.8552,87,NaN
68,mar del plata,AR,-38.0023,-57.5575,89,NaN
87,salalah,OM,17.0151,54.0924,74,NaN
107,dali,CN,25.7000,100.1833,54,NaN
158,gwanda,ZW,-20.9333,29.0000,64,NaN
163,chiredzi,ZW,-21.0500,31.6667,80,NaN
203,gohpur,IN,26.8833,93.6333,46,NaN
251,piet retief,ZA,-27.0071,30.8132,46,NaN


In [15]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {
    "categories" : categories,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hermanus - nearest hotel: Aloe guest house
bredasdorp - nearest hotel: Victoria Hotel
flinders - nearest hotel: Nova Kiama
mar del plata - nearest hotel: Nuevo Ostende
salalah - nearest hotel: Muscat International Hotel
dali - nearest hotel: 风花雪月大酒店
gwanda - nearest hotel: No hotel found
chiredzi - nearest hotel: Nesbitt Arms Hotel
gohpur - nearest hotel: No hotel found
piet retief - nearest hotel: LA Guest House
sattur - nearest hotel: Hotel Raja
morgan city - nearest hotel: Holiday Inn Express & Suites Morgan City - Tiger Island
neuquen - nearest hotel: Hotel Argentina 307
buin - nearest hotel: Hotel Casa Real
requena - nearest hotel: Hotel Avenida
amalapuram - nearest hotel: Homewood Suites
iberia - nearest hotel: Inn of Iberia
baghdad - nearest hotel: AL Bedaa Hotel
gat - nearest hotel: No hotel found
faya - nearest hotel: No hotel found
imphal - nearest hotel: Phou-Oi-Bee Hotel
copiapo - nearest hotel: Hotel Il Riposo


,City,Country,Lat,Lng,Humidity,Hotel Name
15,hermanus,ZA,-34.4187,19.2345,68,Aloe guest house
20,bredasdorp,ZA,-34.5322,20.0403,86,Victoria Hotel
39,flinders,AU,-34.5833,150.8552,87,Nova Kiama
68,mar del plata,AR,-38.0023,-57.5575,89,Nuevo Ostende
87,salalah,OM,17.0151,54.0924,74,Muscat International Hotel
107,dali,CN,25.7000,100.1833,54,风花雪月大酒店
158,gwanda,ZW,-20.9333,29.0000,64,No hotel found
163,chiredzi,ZW,-21.0500,31.6667,80,Nesbitt Arms Hotel
203,gohpur,IN,26.8833,93.6333,46,No hotel found
251,piet retief,ZA,-27.0071,30.8132,46,LA Guest House


In [16]:
%%capture --no-display

# Configure the map plot
ideal_cities_map = hotel_df.hvplot.points("Lng", 
                                          "Lat", 
                                          geo = True, 
                                          tiles = "OSM", 
                                          frame_width = 800, 
                                          frame_height = 600, 
                                          size = "Humidity", 
                                          scale = 1, 
                                          color = "City", 
                                          hover_cols = ["City", "Hotel Name", "Country"])

# Display the map
ideal_cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)